In [22]:
import urllib.request
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", 
                           filename="./data/ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", 
                           filename="./data/ratings_test.txt")



('./data/ratings_test.txt', <http.client.HTTPMessage at 0x29a791f6520>)

In [3]:
train_data = pd.read_table('data/ratings_train.txt')
test_data = pd.read_table('data/ratings_test.txt')

train_data.head()


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
print(train_data.shape, test_data.shape)


(150000, 3) (50000, 3)


In [5]:
train_data.drop_duplicates(subset=['document'], inplace=True)

train_data.isnull().sum()
train_data = train_data.dropna(axis=0)



In [6]:
import re


sample_text = '대체될 문장 입력'
re.sub(r'[^a-zA-Z0-9]', ' ', sample_text)  # 뒤에 ",sample_text" 라고 적으면 sample_text 출력

sample_text = '오류 출력'
re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ',sample_text)



'오류 출력'

In [7]:
train_data['document'].replace('', np.nan, inplace=True)
train_data.dropna(inplace=True)

print(len(train_data))

146182


In [8]:
train_data['document'].isnull().sum()
# train_data['document'].isna().sum() # null 값 출력

0

In [9]:
# train_data 중복값 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data = train_data.dropna(axis=0)

# 한글을 제외한 문자 '' 로 데체 -> 정규 표현식 수행 str.replace('바꿀 문자', '바뀐 문자')
sample_text = '오류 출력'
re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 " "]', '',sample_text) # ^+blank  -> '' 로 수행 가능


# '' -> np.nan으로 변경
train_data['document'].replace('', np.nan, inplace=True,regex=True)

# dropna() -> nan 값으로 변경한 값을 제거
train_data.dropna(inplace=True)

print(len(train_data))


146182


In [10]:
train_data.sort_values(by=['id'])

,id,document,label
46345,33,최고의 영화...정말 최고다..,1
72262,93,조폭마누라 2와 맞먹는.............,0
97347,127,볼만하다..좋다.아주,1
9688,138,한번쯤 볼만한 독특한 프랑스 애미메이션,0
26860,223,재미 무지없었다.공짜로 다시보라고해도 안 봄.,0
...,...,...,...
21109,10278056,매번 더 아름다워지는 영화,1
143537,10278072,에드워드 노튼의 보두앵 4세에 압도되었다. 종교를 가지고 있는 사람으로써 생각할 거...,1
4560,10278099,평점만 믿고 봤는데 결말보고 아렇게 짜증난 영화는 오랜만임,0
92095,10278102,상훈은 외강내유같은 사람이다,1


In [17]:
token= Tokenizer()
token.fit_on_texts(train_data)
x = token.texts_to_sequences(train_data)


X_train = train_data.drop('label', axis=1)
y_train = train_data['label']




In [18]:
word_size = len(token.word_index) + 1

model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              32        
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

path = 'data/model/all/mnist.hdf5'
checkpoint = ModelCheckpoint(path, monitor='val_loss', verbose=0, save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train,y_train,epochs=20,batch_size=32,callbacks=[checkpoint,earlystop])



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).